In [1]:
import numpy as np
import pandas as pd
from statsmodels.api import OLS
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import roc_auc_score

/home/users/magu/.local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# LDL, Height, BMI, diabetes, renal failure, heart attack, asthma, non-melanoma skin cancer
phenos = pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.phe',
                       usecols=['IID','INI20030780','INI50','INI21001','HC221','HC294','HC326','HC382','cancer1060'], 
                       index_col=['IID'],
                       na_values=-9)
phenos[[i for i in phenos.columns if 'INI' not in i]] -= 1
phenos.loc[~phenos.index.duplicated(keep='first')]
phenos.describe()

,HC294,INI50,cancer1060,HC326,HC382,INI20030780,HC221,INI21001
count,488374.000000,500060.000000,488374.000000,488374.000000,488374.000000,3.183660e+05,488374.000000,499520.000000
mean,0.021469,168.419126,0.046743,0.036075,0.129935,1.419452e-17,0.065823,27.433331
std,0.144942,9.276846,0.211088,0.186477,0.336233,9.999906e-01,0.247972,4.801300
min,0.000000,75.000000,0.000000,0.000000,0.000000,-4.615246e+00,0.000000,12.121200
25%,0.000000,161.500000,0.000000,0.000000,0.000000,-6.744867e-01,0.000000,24.141500
50%,0.000000,168.000000,0.000000,0.000000,0.000000,6.957486e-17,0.000000,26.743300
75%,0.000000,175.000000,0.000000,0.000000,0.000000,6.744867e-01,0.000000,29.904100
max,1.000000,209.000000,1.000000,1.000000,1.000000,4.615246e+00,1.000000,74.683700


In [3]:
a,b = 'white_british', 'non_british_white'
with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+a+'.phe', 'r') as f:
    pop = {int(line.split()[0]) for line in f}

with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+b+'.phe', 'r') as f:
    pop2 = {int(line.split()[0]) for line in f}

In [4]:
z='all_beta_center_p001_20190530_500PCs'
npz = np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/'+z+'.npz')
scores = pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+z+'_full.profile',
                       sep='\s+',
                       index_col='IID')
scores = scores.loc[scores.index.isin(phenos.index),:]
phenos = phenos.loc[phenos.index.isin(scores.index),:]

scores.iloc[:,:10].head()

,SCORE_PC1,SCORE_PC2,SCORE_PC3,SCORE_PC4,SCORE_PC5,SCORE_PC6,SCORE_PC7,SCORE_PC8,SCORE_PC9,SCORE_PC10
IID,,,,,,,,,,
-1,103.7650,-56.84110,-329.4330,72.4660,-226.1600,43.4152,77.91040,7.44528,-39.1203,-52.61950
-2,-936.5990,99.50940,-47.4708,-63.9602,-111.8800,-53.3234,-3.12852,-38.45870,114.5300,289.30500
-3,-330.6880,175.75900,201.2250,177.9940,21.2985,-16.4617,88.90270,58.59450,-54.6371,-2.89849
-4,-188.0230,58.26510,-250.1530,-23.0189,-110.5550,57.3824,33.25550,-51.58510,131.0040,176.63200
-5,24.3653,-7.05218,60.2319,74.4917,-301.1660,142.0400,-56.66530,-18.17940,61.7944,159.03100


In [7]:
for p,name in zip([pop,pop2],['White British:','Non-British White:']):
    print(name)
    for phe in map(str,phenos.columns):
        print(phe),
        pcvec = npz['V'][np.where(npz['label_phe_code'] == phe),:].flatten()
        score = scores.loc[scores.index.isin(p),:].sort_index().dot(pcvec).dropna()
        pheno = phenos.loc[phenos.index.isin(p),phe].sort_index().dropna()
        data = pd.concat([score,pheno], axis=1)
        if 'INI' in phe:
            print('r^2={:.4}'.format(data.corr(method='spearman').iloc[0,1]**2))
        else:
            print('AUC={:.4}'.format(roc_auc_score(data.iloc[:,1].astype(bool), data.iloc[:,0])))
    print('')

White British:
HC294 AUC=0.6774
INI50 r^2=0.09414
cancer1060 AUC=0.7659
HC326 AUC=0.8682
HC382 AUC=0.6392
INI20030780 r^2=0.04468
HC221 AUC=0.7357
INI21001 r^2=0.1395

Non-British White:
HC294 AUC=0.5432
INI50 r^2=0.07161
cancer1060 AUC=0.6012
HC326 AUC=0.5759
HC382 AUC=0.5726
INI20030780 r^2=nan
HC221 AUC=0.5895
INI21001 r^2=0.05558



In [6]:
any(npz['label_phe_code']=='INI20030780')

True